In [ ]:
import sys; sys.path.append('../..')
import numpy as np, elastic_rods
from bending_validation import suppress_stdout
from linkage_vis import LinkageViewer

l = elastic_rods.RodLinkage('../../examples/optimized/data/AsymmWingsPointy/1935b524-e979-4340-9245-326f69b6eae0.obj')
mat = elastic_rods.RodMaterial('Rectangle', 20000, 0.3, [12,8], stiffAxis=elastic_rods.StiffAxis.D1)
l.setMaterial(mat)

driver = l.centralJoint()

l.setPerSegmentRestLength(np.loadtxt('../../examples/optimized/data/AsymmWingsPointy/design_parameters.txt'))

jdo = l.dofOffsetForJoint(driver)
fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint
with suppress_stdout(): elastic_rods.compute_equilibrium(l, fixedVars=fixedVars)
view = LinkageViewer(l, width=1024)
view.setCameraParams(((0.4075185454716348, 3.1892642201046715, 0.3099480992441177),
 (0.15364528336486324, 0.2839547329660347, 0.9464474821805594),
 (0.0, 0.0, 0.0)))
view.show()

In [ ]:
from open_linkage import open_linkage
def equilibriumSolver(tgtAngle, l, opts, fv):
    opts.beta = 1e-8
    opts.gradTol = 1e-4
    opts.niter = 100
    opts.useIdentityMetric = False
    return elastic_rods.compute_equilibrium(l, tgtAngle, options=opts, fixedVars=fv)
# open_linkage(l, driver, np.deg2rad(113) - l.averageJointAngle, 50, view, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=20, verbose=10, useTargetAngleConstraint=True);
open_linkage(l, driver, np.deg2rad(113) - l.averageJointAngle, 50, view, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=20, verbose=10, useTargetAngleConstraint=False);

In [ ]:
with suppress_stdout(): elastic_rods.compute_equilibrium(l, 1.0 * externalForces, fixedVars=fixedVars)
view.update()

In [ ]:
from write_render_files import writeRenderFiles, writeActuators
writeRenderFiles(l, 'AsymPointyDataFlat', 'asymPointy_Fab')
writeActuators(l, np.where(torques > 1e-4)[0], directory='AsymPointyDataFlat', name='asymPointy_Fab')

In [ ]:
print("\n".join(map(str, a)), end='', file=open('actuated_joints.txt', 'w'))